In [1]:
from sqlalchemy import create_engine

In [2]:
from sqlalchemy.orm import declarative_base

In [42]:
engine = create_engine('sqlite:///db.sqlite3', echo=True)
engine

# sqlalchemy 에서 데이터베이스 연결을 설정하는 코드
# create_engine 함수는 데이터베이스 연결을 생성하는 함수
# 'sqlite:///db.sqlite3' 은 데이터베이스 파일의 경로를 지정하는 문자열
# /// 는 현재 파일의 위치를 기준으로 한 상대 경로를 나타냅니다.
# echo=True 는 데이터베이스 연결 과정을 콘솔에 출력하는 옵션

Engine(sqlite:///db.sqlite3)

In [43]:
Base = declarative_base() 

# 데이터베이스 모델을 정의하기 위한 기본 클래스를 생성
# 이 기본 클래스는 모든 데이터베이스 모델 클래스의 부모 클래스가 됩니다.

In [44]:
from sqlalchemy import Column, Integer, String

class User(Base): # 모델 클래스는 Base 클래스를 상속받아야 합니다.
    __tablename__ = 'users' # 테이블 이름 지정
    id = Column(Integer, primary_key=True) 
    name = Column(String)
    fullname = Column(String)
    nickname = Column(String)

In [50]:
# 테이블 생성
Base.metadata.create_all(engine)

2025-05-18 13:46:53,555 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-18 13:46:53,556 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2025-05-18 13:46:53,556 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-18 13:46:53,557 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2025-05-18 13:46:53,558 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-18 13:46:53,559 INFO sqlalchemy.engine.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	nickname VARCHAR, 
	PRIMARY KEY (id)
)


2025-05-18 13:46:53,561 INFO sqlalchemy.engine.Engine [no key 0.00073s] ()
2025-05-18 13:46:53,575 INFO sqlalchemy.engine.Engine COMMIT


In [54]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
Session
# sqlalchemy 에서 데이터베이스 세션을 생성하는 코드
# sessionmaker 함수는 데이터베이스 세션을 생성하는 함수
# bind=engine 은 이전에 생성한 데이터베이스 엔진과 연결

# 세션의 주요 기능
# - 데이터베이스 쿼리 실행
# - 객체 추가/수정/삭제
# - 트랜잭션 관리
# - 변경사항 커밋/롤백


sessionmaker(class_='Session', bind=Engine(sqlite:///db.sqlite3), autoflush=True, expire_on_commit=True)

In [55]:
session = Session()
session

In [57]:
new_user = User(name='John', fullname='John Doe', nickname='JD')
new_user


In [58]:
session.add(new_user)

In [59]:
session.commit()

# 세션에 새로운 객체를 추가하고 커밋하는 코드
# session.add(new_user) 는 세션에 새로운 객체를 추가하는 코드
# session.commit() 는 세션에 추가된 객체를 데이터베이스에 저장하는 코드

2025-05-18 13:50:40,290 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-18 13:50:40,292 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?) RETURNING id
2025-05-18 13:50:40,293 INFO sqlalchemy.engine.Engine [generated in 0.00011s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('John', 'John Doe', 'JD')
2025-05-18 13:50:40,300 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?) RETURNING id
2025-05-18 13:50:40,301 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] ('John', 'John Doe', 'JD')
2025-05-18 13:50:40,302 INFO sqlalchemy.engine.Engine COMMIT


In [60]:
session.close()

In [63]:
# 모든 사용자 조회
all_users = session.query(User).all()
all_users

2025-05-18 13:52:10,469 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users
2025-05-18 13:52:10,471 INFO sqlalchemy.engine.Engine [cached since 29.55s ago] ()


[<__main__.User at 0x254c5e7e790>, <__main__.User at 0x254c5e7f390>]

In [66]:
if all_users:
    print("데이터베이스에 데이터가 있습니다.")
    for user in all_users:
        print(user.name, user.fullname, user.nickname)
else:
    print("데이터베이스에 데이터가 없습니다.")

데이터베이스에 데이터가 있습니다.
John John Doe JD
John John Doe JD


In [68]:
first_user = session.query(User).first()
if first_user:
    print("첫 번째 사용자:", first_user.name, first_user.fullname, first_user.nickname)

2025-05-18 13:54:06,407 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users
 LIMIT ? OFFSET ?
2025-05-18 13:54:06,409 INFO sqlalchemy.engine.Engine [cached since 20.29s ago] (1, 0)
첫 번째 사용자: John John Doe JD


In [70]:
# 수정할 사용자 조회(예: nickname으로 조회)
user_to_update = session.query(User).filter_by(nickname='JD').first()

if user_to_update:
    print(f"수정 전: ID: {user_to_update.id}, 이름: {user_to_update.name}, 성: {user_to_update.fullname}, 별명: {user_to_update.nickname}")
    
    # 사용자 정보 수정
    user_to_update.nickname = 'faeqsu10'
    
    # 변경사항 커밋
    session.commit()
    print(f"수정 후: ID: {user_to_update.id}, 이름: {user_to_update.name}, 성: {user_to_update.fullname}, 별명: {user_to_update.nickname}")
    
    # 변경 확인
    updated_user = session.query(User).filter_by(id=user_to_update.id).first()
    if updated_user:
        print(f"DB에서 다시 조회: ID: {updated_user.id}, 이름: {updated_user.name}, 성: {updated_user.fullname}, 별명: {updated_user.nickname}")
else:
    print("닉네임이 'JD'인 사용자를 찾을 수 없습니다.")

2025-05-18 13:58:45,814 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.nickname = ?
 LIMIT ? OFFSET ?
2025-05-18 13:58:45,815 INFO sqlalchemy.engine.Engine [cached since 187.1s ago] ('JD', 1, 0)
수정 전: ID: 1, 이름: John, 성: John Doe, 별명: JD
2025-05-18 13:58:45,818 INFO sqlalchemy.engine.Engine UPDATE users SET nickname=? WHERE users.id = ?
2025-05-18 13:58:45,818 INFO sqlalchemy.engine.Engine [generated in 0.00103s] ('faeqsu10', 1)
2025-05-18 13:58:45,827 INFO sqlalchemy.engine.Engine COMMIT
2025-05-18 13:58:45,830 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-18 13:58:45,833 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.id = ?
2025-05-18 13:58:45,833 INFO sqlalchemy.engine.Engine [generated in 0.00107s] (1,)
수정 후: ID: 1, 이

In [73]:
# 삭제할 사용자 조회 (예: name으로 조회)
user_to_delete = session.query(User).filter_by(name='John').first()

if user_to_delete:
    print(f"삭제 전: ID: {user_to_delete.id}, 이름: {user_to_delete.name}, 성: {user_to_delete.fullname}, 별명: {user_to_delete.nickname}")
    
    # 삭제 실행
    session.delete(user_to_delete)
    session.commit()
    
    print(f"삭제 후: ID: {user_to_delete.id}, 이름: {user_to_delete.name}, 성: {user_to_delete.fullname}, 별명: {user_to_delete.nickname}")
    
    # 삭제 확인
    deleted_user = session.query(User).filter_by(id=user_to_delete.id).first()
    if deleted_user:
        print(f"DB에서 다시 조회: ID: {deleted_user.id}, 이름: {deleted_user.name}, 성: {deleted_user.fullname}, 별명: {deleted_user.nickname}")
else:
    print("이름이 'John'인 사용자를 찾을 수 없습니다.")

2025-05-18 14:00:59,627 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2025-05-18 14:00:59,628 INFO sqlalchemy.engine.Engine [cached since 29.16s ago] ('John', 1, 0)
이름이 'John'인 사용자를 찾을 수 없습니다.


---